In [1]:
from models.attention_series import Grade_regressor
import spectral
import re
import os
import torch
import ast
import numpy as np
from PIL import Image
from torch.utils.tensorboard import SummaryWriter
spectral.settings.envi_support_nonlowercase_params = True

In [2]:
test_list = ['14_A',
            '11_C','16_A','13_B','21_B','36_C','31_A','34_B','56_A',
            '47_C','8_A','19_B','3_A','62_A','33_B','10_A','24_B','60_B',
            '11_B','59_C','10_B','60_C','45_B','8_C','41_A','38_B',
            '57_C','50_C','61_B','37_C']

dataset_path = "E:\\d盘备份\\近红外部分"

#for test: 14 19 54
mask_rgb_values = [[255,242,0],[34,177,76],[255,0,88]]
pool = torch.nn.AvgPool2d(3,3)
test_data = []
for id in test_list:
    pixel_list = []
    imgid, sampleid = id.split('_')
    sampleid = ord(sampleid) - 65
    img_data = spectral.envi.open(dataset_path+"\\spectral_data\\{}-Radiance From Raw Data-Reflectance from Radiance Data and Measured Reference Spectrum.bip.hdr".format(imgid))
    gt = ast.literal_eval(img_data.metadata['gt_TFe'])
    img_data = torch.Tensor(img_data.asarray()/6000)[:,:,:]
    img_data = pool(img_data.permute(2,0,1)).permute(1,2,0)
    mask = np.array(Image.open(dataset_path+"\\spectral_data\\{}-Radiance From Raw Data-Reflectance from Radiance Data and Measured Reference Spectrum.bip.hdr_mask.png".format(imgid)))
    row, col, _ = img_data.shape
    for r in range(row):
        for c in range(col):
            if mask[r*3+1,c*3+1].tolist() == mask_rgb_values[sampleid]:
                pixel_list.append(img_data[r,c].unsqueeze(0))

    pixel_list = torch.cat(pixel_list, dim=0)
    test_data.append({
        "tensor": pixel_list.to("cuda"),
        "gt": torch.Tensor([gt[sampleid]]).to("cuda")
    })


In [3]:
re_prefix = "(TSI)TFe_0.93_unbalanced"

In [4]:
sum_writer = SummaryWriter("./runs/{}".format(re_prefix))

In [5]:
ckpt_path = "D:\\source\\repos\\Pixel-wise-hyperspectral-feature-classification-experiment\\ckpt"

In [6]:
for i in range(1,10000000000):
    state_dict_path = ckpt_path+'/'+re_prefix+'_'+str(i*1000)+".pt"
    if os.path.exists(state_dict_path):
        print(ckpt_path+'/'+re_prefix+'_'+str(i*1000)+".pt")
        # 导入模型
        model = Grade_regressor().to("cuda").eval()
        model.load_state_dict(torch.load(state_dict_path))
        # 算RE
        re = 0
        err_count = 0
        for d in test_data:
            data = d['tensor']
            gt = d['gt']
            with torch.no_grad():
                len = data.shape[0]
                cur = 0
                pixelwise_prediction = []
                while cur + 100 <= len:
                    pixelwise_prediction.append(model(data[cur:cur+100]).to("cpu"))
                    cur += 100
                    torch.cuda.empty_cache()

                if cur != len:
                    pixelwise_prediction.append(model(data[cur:len]).to("cpu"))

                pixelwise_prediction = torch.cat(pixelwise_prediction, dim=0)
            prediction = pixelwise_prediction.mean()*100
            re += 100*torch.abs(gt - prediction)/gt
            err_count += 1

        # 写入summary writer
        avg_re = re/(err_count)

        sum_writer.add_scalar(tag='RE',
                        scalar_value=avg_re,
                        global_step=i*1000
                    )

    else:
        break

D:\source\repos\Pixel-wise-hyperspectral-feature-classification-experiment\ckpt/(TSI)TFe_0.93_unbalanced_1000.pt
D:\source\repos\Pixel-wise-hyperspectral-feature-classification-experiment\ckpt/(TSI)TFe_0.93_unbalanced_2000.pt
D:\source\repos\Pixel-wise-hyperspectral-feature-classification-experiment\ckpt/(TSI)TFe_0.93_unbalanced_3000.pt
D:\source\repos\Pixel-wise-hyperspectral-feature-classification-experiment\ckpt/(TSI)TFe_0.93_unbalanced_4000.pt
D:\source\repos\Pixel-wise-hyperspectral-feature-classification-experiment\ckpt/(TSI)TFe_0.93_unbalanced_5000.pt
D:\source\repos\Pixel-wise-hyperspectral-feature-classification-experiment\ckpt/(TSI)TFe_0.93_unbalanced_6000.pt
D:\source\repos\Pixel-wise-hyperspectral-feature-classification-experiment\ckpt/(TSI)TFe_0.93_unbalanced_7000.pt
D:\source\repos\Pixel-wise-hyperspectral-feature-classification-experiment\ckpt/(TSI)TFe_0.93_unbalanced_8000.pt
D:\source\repos\Pixel-wise-hyperspectral-feature-classification-experiment\ckpt/(TSI)TFe_0.93_un